### Estid Lozano
### David Herrera

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
def getX_Y(csv_name, scale=True):
    df = pd.read_csv(csv_name)
    X, y = df.values[:,:-1].astype(float), df.values[:,-1]
    if scale:
        X -= np.min(X, axis=0)
        Xmax = np.max(X, axis=0)
        Xmax[Xmax == 0] = 1 # avoid division by zero
        X /= Xmax
    classes = np.unique(y)
    yDict = dict(zip(classes, np.identity(len(classes), dtype=int).tolist()))
    yOnes = np.array([yDict[i] for i in y])
    return  X, y.reshape(len(y), 1), yOnes, classes
    
def softmax(x):
    xp = x - max(x)
    return np.exp(xp) / np.sum([np.exp(xi) for xi in xp])

'''
    Computes the Jacobian (matrix of derivatives) of a softmax layer given the outputs produced by that layer.
    Attention: Do NOT provide the net values of the softmax layer here but the probability vector it produced.
'''
def softmax_deriv(z):
    n = len(z)
    d = np.zeros((n, n))
    for i in range (n):
        for j in range(i):
            d[i,j] = d[j,i] = -(z[i] * z[j])
        d[i,i] = z[i] * (1-z[i])
    return d

def cross_entropy_error(y, o):
    return np.abs(np.dot(y.T,np.log(np.maximum(10**-20, o))))

def cross_entropy_error_derivative(y, o):
    return -y/np.maximum(10**-20,o)

def get_activation(name):
    if name == "linear":
        return lambda x: x
    if name == "relu":
        return lambda x: np.maximum(0, x)
    if name == "softmax":
        return softmax

def get_derivative(name):
    if name == "linear":
        return lambda x: np.eye(len(x))
    if name == "relu":
        return lambda x: np.diag((x > 0).astype(int).T[0])
    if name == "softmax":
        return softmax_deriv

# Exercise 1

Let q $\in \mathbb{N}$ and f:$\mathbb{R} \rightarrow \mathbb{R}^q$ and g:$\mathbb{R}^q \rightarrow \mathbb{R}$ two functions.

$\LARGE{ \frac{\delta g \circ f}{\delta x} \stackrel{def}{=} \frac{g(f(x))}{\delta x} = \sum^q_{i=1}\frac{\delta g}{\delta f_i} \cdot \frac{\delta f_i}{\delta x} }$

where $f_i$ refers to the i-th component in the output of $f$.
Use this to show that

$\LARGE{ \frac{\delta \varepsilon_x}{\delta z^l} = \sum^{n_{l+1}}_{k=1} \frac{\delta \varepsilon_x}{\delta net^{l+1}_k} \cdot \frac{\delta net^{l+1}_k}{\delta z^l} }$

where $net^l_i$ is the net value of the i-th neuron in layer l. Conclude that

$\LARGE{ \frac{\delta \varepsilon_x}{\delta net^l_j} = \frac{\delta f^l(net^l_j)}{\delta net^l_j} \cdot \sum^{n_{l+1}}_{k=1} \frac{\delta \varepsilon_x}{\delta net^{l+1}_k} \cdot \frac{\delta net^{l+1}_k}{\delta f^l(net^l_j)} }$

**Solution**

Knowing that $\LARGE{ \frac{g(f(x))}{\delta x} = \sum^q_{i=1}\frac{\delta g}{\delta f_i} \cdot \frac{\delta f_i}{\delta x} }$

We have $\LARGE{ \frac{\delta \varepsilon_x}{\delta net^l_j} = \frac{\delta \varepsilon_x}{\delta z^l} \cdot \frac{\delta z^l}{\delta net^l_j} }$

Then, we replace $\LARGE{ \frac{\delta \varepsilon_x}{\delta z^l} = \sum^{n_{l+1}}_{k=1} \frac{\delta \varepsilon_x}{\delta net^{l+1}_k} \cdot \frac{\delta net^{l+1}_k}{\delta z^l} }$

So we get $\LARGE{ \frac{\delta \varepsilon_x}{\delta net^l_j} = \frac{\delta z^l}{\delta net^l_j} \cdot \sum^{n_{l+1}}_{k=1} \frac{\delta \varepsilon_x}{\delta net^{l+1}_k} \cdot \frac{\delta net^{l+1}_k}{\delta z^l} }$

Finally, we know that $\LARGE{ z^l= f^l(net^l_j) }$

Getting $\LARGE{ \frac{\delta \varepsilon_x}{\delta net^l_j} = \frac{\delta f^l(net^l_j)}{\delta net^l_j} \cdot \sum^{n_{l+1}}_{k=1} \frac{\delta \varepsilon_x}{\delta net^{l+1}_k} \cdot \frac{\delta net^{l+1}_k}{\delta f^l(net^l_j)} }$

# Exercise 2

We now implement the neural network incrementally by extending
the *ANNClassifier* class, which already comes with methods for setup and prediction. The classifier is set up with an array of descriptions of the hidden layer (one entry for each hidden layer). Your task is now to establish the learning process through back-propagation. Note that the last layer is, by definition, a softmax layer for classification or a linear layer for regression problems; in both cases, $\delta^{h+1}=o-y$.

**2.1.** Implement a method *compute_net_gradients(z, delta_of_output_layer)* that computes a list containing the net gradients $\delta^l$ for all layers (the output layer net gradient should also be contained). This is line 16 in the pseudo code.

**2.2.** Implement a method *update_weights_and_biases(z, delta)* that realizes lines 17-22 in the pseudo code.

**2.3.** Implement a method *step(x, y)* that updates the whole model for the instance x with its label y, i.e. that realizes the whole procedure of lines 9-22 for this instance.

**2.4.** Implement a method *train(X, y)* that takes a set of given training data with its labels, invokes *setup* with the appropriate arguments, and realizes the outer loop (lines 7 + 8)
of the algorithm relying on the method *step* for each instance.

In [ ]:
class ANNClassifier:
    def __init__(self, architecture, learning_rate = 1.0, max_iter = 10):
        self.architecture = architecture
        self.h = len(architecture)
        self.n = [layer["units"] for layer in architecture]
        self.f = [get_activation(layer["activation"]) for layer in architecture]
        self.fd = [get_derivative(layer["activation"]) for layer in architecture]
        self.f.append(get_activation("softmax"))
        self.fd.append(get_derivative("softmax"))
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        
    def reset(self, input_dimension, labels, seed = 0):
        self.labels = labels
        layer_sizes = [input_dimension] + self.n + [len(labels)]
        self.b, self.W = [], []
        np.random.seed(seed)
        for l in range(self.h):
            self.b.append(np.random.uniform(low=-0.01,high=0.01,size=(layer_sizes[l + 1], 1)))
            self.W.append(np.random.uniform(low=-0.01,high=0.01,size=(layer_sizes[l], layer_sizes[l + 1])))
        self.b.append(np.random.uniform(low=-0.01,high=0.01,size=(layer_sizes[-1], 1)))
        self.W.append(np.random.uniform(low=-0.01,high=0.01,size=(layer_sizes[-2], layer_sizes[-1])))
    
    def forward_phase(self, x):
        z = [x.reshape(len(x), 1)]
        for l in range(self.h + 1):
            net = np.dot(self.W[l].T, z[l]).reshape(len(self.b[l]), 1) + self.b[l]
            if any(np.isnan(net)):
                raise Exception("Found nan-value in net: " + str(net))
            zl = self.f[l](net)
            if any(np.isnan(zl)):
                raise Exception("Found nan-value in z-vector after applying activation function " + str(self.f[l]) + " to arguments " + str(net) + ": " + str(zl) + ". These is the complete list of computed z-valuies: " + str(z))
            z.append(zl)
        return z
    
    def step(self, x, y):
        z = self.forward_phase(x)
        oi = z[-1]
        deltas = self.compute_net_gradients(z, oi - y.reshape(len(y), 1))
        self.update_weights_and_biases(z, deltas)
        
    def train(self, X, y):
        self.reset(X.shape[1], list(np.unique(y,axis=0)))
        # Setup
        for t in range(self.max_iter + 1):
            random_indexes = list(range(len(X)))
            random.shuffle(random_indexes)
            for i in random_indexes:
                self.step(X[i], y[i])
    
    def compute_net_gradients(self, z, delta_of_output_layer):
        # Backpropagation Phase
        deltas = [None] * (self.h + 2)
        deltas[-1] = delta_of_output_layer
        for l in range(self.h, 0, -1):
            deltas[l] = np.dot(self.fd[l](z[l]), np.dot(self.W[l], deltas[l + 1]))
        return deltas
    
    def update_weights_and_biases(self, z, deltas):
        # Gradient Descent Step
        hr = range(self.h + 1)
        grads_W = [np.dot(z[l], deltas[l + 1].T) for l in hr]
        grads_b = [deltas[l + 1] for l in hr]
        for l in range(self.h + 1):
            self.W[l] -= self.learning_rate * grads_W[l]
            self.b[l] -= self.learning_rate * grads_b[l]
    
    def predict(self, X):
        return [self.labels[np.argmax(self.forward_phase(x)[-1])] for x in X]

# Exercise 3

**3.1.** Train 10 networks with 1 to 10 units in a single hidden layer on the iris dataset. What
do you observe? Plot the in-sample prediction results.

In [ ]:
%matplotlib inline
X, y, yOnes, classes = getX_Y("iris.csv")

for l in range(1, 11):
    architecture = [
        {
            "units": l,
            "activation": "linear"
        }
    ]
    ann = ANNClassifier(architecture, learning_rate = 0.01, max_iter = 10)
    
    ann.train(X, yOnes)

    y_hat = np.array([classes[list(ta).index(1)] for ta in ann.predict(X)]).reshape(len(X), 1)

    mask_correct = (y_hat == y).T[0]
    fig, ax = plt.subplots()
    plt.title("Units:" + str([layer["units"] for layer in architecture]))
    markers = ["o", "^", "x"]
    for i, label in enumerate(np.unique(y)):
        indices_pred = (y == label)[:,0]
        i1 = indices_pred & mask_correct
        i2 = indices_pred & ~mask_correct
        ax.scatter(X[i1, 2], X[i1, 1], color="limegreen", marker=markers[i])
        ax.scatter(X[i2, 2], X[i2, 1], color="red", marker=markers[i])
    plt.show()

**3.2.** Download the MNIST dataset from *https://www.openml.org/data/get_csv/52667/mnist_784.arff*.

In [ ]:
%matplotlib inline
def visualizeLearningProcess(ann, X_train, y_train, X_test, y_test, max_epochs = 100, plot_step_size = 100,title = ""):
    d = X_train.shape[1]
    ann.reset(d, list(np.unique(y_train,axis=0)))
    
    lc = []
    fig, ax = plt.subplots()
    queue = []
    epochs = 0
    steps = max_epochs * X_train.shape[0]
    for t in tqdm(range(steps)):
        
        # get example
        if len(queue) == 0:
            epochs += 1
            queue = random.sample(range(len(X_train)), len(X_train))
            
        # update model
        index = queue[0]
        del queue[0]
        x = X_train[index].reshape(d, 1)
        y = y_train[index]
        ann.step(x, y)
        
        if t % plot_step_size == 0:
            # compute error rate
            y_hat = np.array([classes[list(ta).index(1)] for ta in ann.predict(X_test)]).reshape(len(X_test), 1)
            error_rate = np.count_nonzero(y_hat != y_test) / len(y_hat)
            lc.append(error_rate)

            # update learning curve
            ax.clear()
            ax.plot(plot_step_size * np.array(range(len(lc))), lc)
            ax.set_ylim([0, 1.01])
            ax.axhline(min(lc), linestyle="--", color="red")
            ax.set_xlabel("Training points")
            ax.set_title(title)
            fig.canvas.draw()

def visualizeMNISTPredictions(ann, classes, num_predictions = 100):
    X = dfMNIST.values[:,:-1].astype(float)
    cols = 10
    rows = int(np.ceil(num_predictions / cols))
    fig, ax = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    for index in range(num_predictions):
        a = ax[int(np.floor(index / cols)), index % cols]
        a.imshow(X[index].reshape(28, 28), cmap="gray_r")
        a.set_xticks([])
        a.set_yticks([])
        prediction = classes[list(ann.predict([X[index]])[0]).index(1)]
        a.set_title("Prediction: " + str(prediction))
    fig.tight_layout()
    plt.show()

X, y, yOnes, classes = getX_Y("mnist.csv", False)

Try 10 networks with one hidden layer and between 1 and 400 units in this layer. Try learning rates {$1,10^{-1},...,10^{-4}$} and report your learning curves.

In [ ]:
sqrt9of400 = 400**(1/9.0)
units = [int(round(sqrt9of400**i)) for i in range(10)]
learning_rates =  [10**-i for i in range(5)]
for l in units:
    for learning_rate in learning_rates:
        architecture = [
            {
                "units": l,
                "activation": "linear"
            }
        ]
        ann = ANNClassifier(architecture, learning_rate = learning_rate, max_iter = 10)
        visualizeLearningProcess(ann,X[:-10000],yOnes[:-10000],X[-10000:],y[-10000:],max_epochs=2,plot_step_size=100,title="units={} rate={}".format(l, learning_rate))

In addition, also try some other configurations with more hidden layers. In this exercise, never user any of the last 10000 instances during the training process.

Which architecture achieves best performance? Plot the predictions with the respective function for at least 100 instances in the test fold (last 10000 instances).

In [ ]:
archs = [[8, 16, 8], [8, 8, 16]]
for arch in archs:
    architecture = [
        {
            "units": u,
            "activation": "linear"
        }
        for u in arch
    ]
    ann = ANNClassifier(architecture, learning_rate = 0.01, max_iter = 10)
    visualizeLearningProcess(ann,X[:-10000],yOnes[:-10000],X[-10000:],y[-10000:],max_epochs=2,plot_step_size=100)
    visualizeMNISTPredictions(ann, classes)

Also observe whether scaling the data into the [0,1] interval (dividing by 255) changes
the algorithm behavior.

In [ ]:
XpseudoScaled = X / 255
archs = [[8, 16, 8], [8, 8, 16]]
for arch in archs:
    architecture = [
        {
            "units": u,
            "activation": "linear"
        }
        for u in arch
    ]
    ann = ANNClassifier(architecture, learning_rate = 0.01, max_iter = 10)
    visualizeLearningProcess(ann,XpseudoScaled[:-10000],yOnes[:-10000],XpseudoScaled[-10000:],y[-10000:],max_epochs=2,plot_step_size=100)
    visualizeMNISTPredictions(ann, classes)